In [122]:
import pandas as pd
from skbio import TreeNode
from os.path import join, basename
from collections import OrderedDict
from os import makedirs
from glob import glob
from shutil import copytree

# Prepare GWCodeML scripts and file

This notebook reads in the set of calculated per-clade genome-wide symbiont trees, reroots them based on the complete marker gene-based symbiont supertree, and identifies the branches of the symbiont tree leading to monophyletic clades of symbiont genomes from target host species, if they exist.

Uses this information to creat per-clade input files to GWCodeML

In [2]:
trees_dir = '../data/renamed/symbiont_trees/'

In [3]:
def count_clade_hosts(tree, focal_host):
    '''
    
    '''
    tips = [x.name for x in tree.tips()]
    spp = [x[0:6] for x in tips]
    focal_tips = sum([x == focal_host for x in spp])
    if focal_tips == 0:
        return(None)
    
    tree.assign_ids()
    clade_counts = {}
    for clade in tree.postorder():
        if clade.is_tip():
            clade_tips = [clade.name]
        else:
            clade_tips = [x.name for x in clade.tips()]
        clade_spp = [x[0:6] for x in clade_tips]
        clade_focal_tips = sum([x == focal_host for x in clade_spp])
        clade_counts[clade.id] = {'id': clade.id,
                                  'tips': len(clade_tips),
                                  'focal_tips': clade_focal_tips,
                                  'prop_of_foci': clade_focal_tips / focal_tips,
                                  'focal_prop': clade_focal_tips / len(clade_tips)}
    
    clade_df = pd.DataFrame.from_dict(clade_counts, orient='index')
    
    return(clade_df)
    

In [4]:
tree_fps = glob(join(trees_dir, '*.nwk'))

In [5]:
trees = {}

In [6]:
trees = {}
dfs = []
focus = 'homsap'

for tree_fp in tree_fps:
    tree = TreeNode.read(tree_fp, 
                         convert_underscores=False)
    cluster = int(basename(tree_fp).split('.')[0].split('_')[1])
    cluster_df = count_clade_hosts(tree, focus)
    if cluster_df is not None:
        trees[cluster] = tree
        cluster_df['cluster'] = cluster
        dfs.append(cluster_df)

## Root trees

Root subtrees based on rooting position in whole marker gene tree

In [14]:
super_tree_fp = '../data/renamed/gtdbtk.bac120.user_msa.fasta.treefile.renamed.tre'

super_tree = TreeNode.read(super_tree_fp,
                           convert_underscores=False)

In [36]:
def reroot(target, source):
    target_outgroups = []
    for child in target.children:
        if child.is_tip():
            target_outgroups.append(child.name)

    for child in source.children:
        if child.is_tip():
            if child.name in target_outgroups:
                return(target)
            else:
                return(target.root_at(target.find(child.name).parent))
        tips = [x.name for x in child.tips()]
        target_node = target.lowest_common_ancestor(tips)
        if len([x for x in target_node.tips()]) == len(tips):
            return(target.root_at(target_node))
    raise ValueError

In [37]:
rerooted = {}

for clade in trees:
    tree = trees[clade]
    super_clade = super_tree.lowest_common_ancestor([x.name for x in tree.tips()])
    try:
        rerooted[clade] = reroot(tree, super_clade)
    except:
        print("Failure on clade %s\n" % clade)
#         print("Target tree:\n")
#         print(tree.ascii_art())
#         print("Source tree:\n")
#         print(super_clade.ascii_art())

Failure on clade 30

Failure on clade 73



## Find and mark branches

In [65]:
host_dfs = {}
target_genera = ['hom', 'pan']

for clade in rerooted:
    tree = rerooted[clade]
    tips = [x.name for x in tree.tips()]
    gen = []
    spp = []
    for tip in tips:
        gen.append(tip[0:3])
        spp.append(tip[3:6])
    clade_df = pd.DataFrame.from_dict({'Tip': tips,
                                       'Genus': gen,
                                       'Species': spp})
    clade_df['Clade'] = clade
    clade_df['Branch'] = 0
    host_dfs[clade] = clade_df
    
    for i, target in enumerate(target_genera):
        target_tips = list(clade_df.loc[clade_df['Genus'] == target, 'Tip'])
        if len(target_tips) < 1:
            continue
        if len(target_tips) == 1:
            clade_df.loc[clade_df['Tip'] == target_tips[0], 'Branch'] = i + 1
            continue
        target_node = tree.lowest_common_ancestor(target_tips)
        lca_tips = [x.name for x in target_node.tips()]
        if len(lca_tips) == len(target_tips):
            clade_df.loc[clade_df['Tip'].isin(target_tips), 'Branch'] = i + 1
    
    

In [66]:
host_dfs

{10:           Tip Genus Species  Clade  Branch
 0  homsap3623   hom     sap     10       0
 1  homsap3621   hom     sap     10       0
 2  homsap3622   hom     sap     10       0
 3  homsap3620   hom     sap     10       0
 4  homsap3619   hom     sap     10       0
 5  homsap3618   hom     sap     10       0
 6  prover0048   pro     ver     10       0
 7  prover0049   pro     ver     10       0
 8  alosen0026   alo     sen     10       0,
 100:           Tip Genus Species  Clade  Branch
 0  homsap0516   hom     sap    100       1
 1  panpan0185   pan     pan    100       2
 2  panpan0186   pan     pan    100       2
 3  pantro0129   pan     tro    100       2
 4  pantro0127   pan     tro    100       2
 5  pantro0128   pan     tro    100       2
 6  gorgor0025   gor     gor    100       0,
 167:            Tip Genus Species  Clade  Branch
 0   pantro0231   pan     tro    167       0
 1   pantro0232   pan     tro    167       0
 2   pansch0352   pan     sch    167       0
 3   pansch0

## Write output files

In [69]:
ctl_fp = '/home/jgs286/git_sw/GWideCodeML/example_data/gwcodeml.ctl'
ctl = ''
with open(ctl_fp, 'r') as f:
    for line in f:
        ctl += line

In [72]:
print(ctl)

      seqfile = seq_file.phy * sequence data filename
     treefile = tree.nwk      * tree structure file name
      outfile = out_name           * main result file name

        noisy = 0  * 0,1,2,3,9: how much rubbish on the screen
      verbose = 0  * 0: concise; 1: detailed, 2: too much
      runmode = 0  * 0: user tree;  1: semi-automatic;  2: automatic
                   * 3: StepwiseAddition; (4,5):PerturbationNNI; -2: pairwise

      seqtype = 1  * 1:codons; 2:AAs; 3:codons-->AAs
    CodonFreq = 2  * 0:1/61 each, 1:F1X4, 2:F3X4, 3:codon table
       aaDist = 0  * 0:equal, +:geometric; -:linear, 1-6:G1974,Miyata,c,p,v,a

   aaRatefile = dat/wag.dat  * only used for aa seqs with model=empirical(_F)
                   * dayhoff.dat, jones.dat, wag.dat, mtmam.dat, or your own

        model = 2  * models for codons:
                       * 0:one, 1:b, 2:2 or more dN/dS ratios for branches
                   * models for AAs or codon-translated AAs:
                       * 0:poiss

In [123]:
ctl_dict = OrderedDict({'seqfile': 'seq_file.phy',
            'treefile': 'tree.nwk',
            'outfile': 'out_name',
            'noisy': 0,
            'verbose': 0,
            'runmode': 0,
            'seqtype': 1,
            'CodonFreq': 2,
            'aaDist': 0,
            'aaRatefile': 'dat/wag.dat',
            'model': 2,
            'NSsites': 2,
            'icode': 0,
            'fix_kappa': 0,
            'kappa': 3,
            'fix_omega': 0,
            'omega': .4,
            'fix_alpha': 1,
            'alpha': 0,
            'Malpha': 0,
            'ncatG': 3,
            'clock': 0,
            'getSE': 0,
            'RateAncestor': 0,
            'Small_Diff': '.5e-6'
})

In [136]:
makedirs(project_dir, exist_ok=True)



In [138]:
aln_dir = '/workdir/jgs/projects/primate_codiv/data/renamed/per_gene_alignments'
project_dir = '/home/jgs286/gwcodeml'
cmd_stub = ('gwidecodeml -model {model} '
            '-work_dir {work_dir} '
            '-cds {cds} '
            '-tree {tree} '
            '-dnds '
            '-branch {branch} '
            '-p {p}')

makedirs(project_dir, exist_ok=True)

cmds = []
for clade in rerooted:
    clade_dir = join(project_dir, 'clade_{0:03d}'.format(clade))
    clade_aln_dir = join(aln_dir, 'cluster_{0:03d}_genes'.format(clade))
    
    # copy alignments
#     copytree(clade_aln_dir, clade_dir, dirs_exist_ok=True)
    
    clade_tree = rerooted[clade]
    clade_tree.name = None
    clade_branch_df = host_dfs[clade]
    
    # write branch fp
    branch_fp = join(clade_dir, '{0:03d}_branch.csv'.format(clade))
    clade_branch_df[['Tip','Branch']].to_csv(branch_fp, header=False, index=False)
    
    # write tree fp
    tree_fp = join(clade_dir, '{0:03d}_tree.nwk'.format(clade))
    clade_tree.write(tree_fp)
    
    # write ctl file
    clade_ctl = ctl_dict.copy()
    clade_ctl['outfile'] = 'clade_{0:03d}_out'.format(clade)
    with open(join(clade_dir, 'gwcodeml.ctl'), 'w') as f:
        for item in ctl_dict:
            f.write('{0} = {1}\n'.format(item, ctl_dict[item]))
    
    # write command
    cmd = cmd_stub.format(model='BM',
                          work_dir=clade_dir,
                          cds='.fa',
                          tree=tree_fp,
                          branch=branch_fp,
                          p=64)
    cmd_fp = join(clade_dir, 'run_gwcodeml.{0:03d}.sh'.format(clade))
    
    cmds.append('cd clade_{0:03d}'.format(clade))
    cmds.append('bash run_gwcodeml.{0:03d}.sh'.format(clade))
    cmds.append('cd ..')
    
    with open(cmd_fp, 'w') as f:
        f.write(cmd)
    
with open(join(project_dir, 'run_all.sh'), 'w') as f:
    f.write('\n'.join(cmds))